In [93]:
import pandas as pd
import dataframe_image as dfi
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm

df1 = pd.read_csv('bench1_perfs.csv', sep=',', names=['duration', 'id', 'k', 'size2', 'nb_items', 'nb_ranks'])
df1 = df1.groupby(['id', 'k', 'size2', 'nb_items', 'nb_ranks'])['duration'].mean()
df1 = df1.rename('PIM').to_frame()

df2 = pd.read_csv('bench2_perfs.csv', sep=',', names=['duration', 'id', 'k', 'size2', 'nb_items'])
df2 = df2.groupby(['id', 'k', 'size2', 'nb_items'])['duration'].mean()
df2 = df2.rename('SyncCache').to_frame()

df = df1.join(df2)
df['PIM_Speedup'] = df['SyncCache'] / df['PIM']
print(df)

                                        PIM  SyncCache  PIM_Speedup
id     k size2 nb_items  nb_ranks                                  
fpr    8 30    10000000  6         0.008830   0.000020     0.002265
                         8         0.009974   0.000020     0.002005
               100000000 6         0.350612   0.013420     0.038276
                         8         0.429230   0.013420     0.031265
         31    10000000  6         0.004792   0.000000     0.000000
...                                     ...        ...          ...
weight 8 32    100000000 8         0.043212  13.461600   311.523567
         33    10000000  6         0.168481  27.344417   162.299320
                         8         0.085210  27.344417   320.907774
               100000000 6         0.168467  26.757100   158.826480
                         8         0.085041  26.757100   314.637275

[80 rows x 3 columns]


In [155]:
style = [
   {'selector': '',
    'props': [
        ('font-family', 'monospace'),
        ('border', '2px solid #e0e0e8'),
        ('vertical-align', 'top'),
        ('color', '#343434'),
        ('font-size', '13px'),
    ]
    },
    {'selector': 'th',
    'props': [
        ('font-family', 'monospace'),
        ('border', '2px solid #e0e0e8'),
        ('text-align', 'right !important'),
        ('vertical-align', 'top'),
        ('padding', '5px'),
        ('color', '#343434'),
    ]
    }]

In [156]:
better_vmax = {
    'lookup': 5.0,
    'init': 50.0,
    'insert': 5.0,
    'weight': 500.0,
    'fpr': 0.0,
}

do_save = True

def plot_table(function_exported):
    norm_better = matplotlib.colors.Normalize(vmin=1.0, vmax=better_vmax[function_exported], clip=True)
    mapper_better = cm.ScalarMappable(norm=norm_better, cmap=cm.YlGn)

    norm_worse = matplotlib.colors.Normalize(vmin=0.0, vmax=1.0, clip=True)
    mapper_worse = cm.ScalarMappable(norm=norm_worse, cmap=cm.RdPu_r)

    def color_bg(v):
        result = []
        for value in v.values:
            if value >= 1.0:
                result.append(mapper_better.to_rgba(value))
            else:
                result.append(mapper_worse.to_rgba(value))
        return list(map(lambda e: 'background-color: ' + matplotlib.colors.to_hex(e), result))

    sub_df = df.xs(function_exported, level='id', drop_level=False)

    s = sub_df
    if function_exported == 'fpr':
        s = s.drop('PIM_Speedup', axis=1).style.set_table_styles(style) \
            .format({'PIM': '{:,.5f}', 'SyncCache': '{:,.5f}'}) \
            .background_gradient(cmap=cm.GnBu, vmin=0.0, vmax=0.5)
    else:
        s = s.style.apply(color_bg, subset=['PIM_Speedup']).set_table_styles(style) \
            .format({'PIM': '{:,.3f} s', 'SyncCache': '{:,.3f} s', 'PIM_Speedup': '{:,.3f}'})

    if do_save:
        dfi.export(s, function_exported + '.png', dpi=150)
    
    return s

In [157]:
plot_table('init')

In [158]:
plot_table('insert')

In [159]:
plot_table('lookup')

In [160]:
plot_table('weight')

In [161]:
plot_table('fpr')